<a href="https://colab.research.google.com/github/Jaise152/Modern-Natural-Language-Processing-in-Python/blob/main/CNN_for_NLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install beautifulsoup4

In [2]:
import pandas as pd
import numpy as np
import re
import math
from bs4 import BeautifulSoup
from google.colab import drive 

In [3]:
try:
  %tensorflow_version 2.x
except Exception:
  pass
import tensorflow as tf
from tensorflow.keras import layers
import tensorflow_datasets as tfds

In [4]:
drive.mount("/content/drive/")

Mounted at /content/drive/


In [5]:
cols = ['sentiment', 'id', 'date', 'query', 'user','text']

In [6]:
test_data = pd.read_csv(
   "/content/drive/MyDrive/Colab Notebooks/Modern-Natural-Language-Processing-in-Python/test.csv",
    header = None,
    names = cols,
    engine = 'python',
    encoding = 'latin1'
)

In [7]:
train_data = pd.read_csv(
   "/content/drive/MyDrive/Colab Notebooks/Modern-Natural-Language-Processing-in-Python/train.csv",
    header = None,
    names = cols,
    engine = 'python',
    encoding = 'latin1'
)

In [8]:
test_data.tail(3)

,sentiment,id,date,query,user,text
495,4,14074,Sun Jun 14 04:36:34 UTC 2009,latex,iamtheonlyjosie,Ahhh... back in a *real* text editing environm...
496,0,14075,Sun Jun 14 21:36:07 UTC 2009,iran,plutopup7,"Trouble in Iran, I see. Hmm. Iran. Iran so far..."
497,0,14076,Sun Jun 14 21:36:17 UTC 2009,iran,captain_pete,Reading the tweets coming out of Iran... The w...


In [9]:
test_data.drop(['id','date','query','user'],
          axis = 1,
          inplace = True)

In [10]:
train_data.drop(['id','date','query','user'],
          axis = 1,
          inplace = True)

In [11]:
BeautifulSoup(test_data['text'][0], 'lxml',).get_text()

'@stellargirl I loooooooovvvvvveee my Kindle2. Not that the DX is cool, but the 2 is fantastic in its own right.'

In [12]:
re.sub(r"@[A-Z a-z 0-9]+", ' ', test_data['text'][0])

' . Not that the DX is cool, but the 2 is fantastic in its own right.'

In [13]:
re.sub(r"https?://[A-Za-z0-9./]+", ' ', test_data['text'][0])

'@stellargirl I loooooooovvvvvveee my Kindle2. Not that the DX is cool, but the 2 is fantastic in its own right.'

In [14]:
re.sub(r"[^A-Za-z0-9.!?]", ' ', test_data['text'][0])

' stellargirl I loooooooovvvvvveee my Kindle2. Not that the DX is cool  but the 2 is fantastic in its own right.'

In [24]:
def clean_tweet(tweet):
  tweet = re.sub(r"@[A-Z a-z 0-9]+", ' ', tweet) #Removing mentiones
  tweet = re.sub(r"https?://[A-Za-z0-9./]+", ' ', tweet) #Removing urls
  tweet = re.sub(r"[^A-Za-z0-9.!?]+", ' ', tweet) #Removing all unwanted characters
  tweet = re.sub(r" +", ' ', tweet)
  return tweet

In [16]:
clean_tweet_test = [clean_tweet(tweet) for tweet in test_data.text ]

In [17]:
clean_tweet_train = [clean_tweet(tweet) for tweet in train_data.text ]

In [18]:
test_data['sentiment'].unique()

array([4, 0, 2])

In [19]:
train_data['sentiment'].unique()

array([0, 4])

In [20]:
data_labels = train_data.sentiment.values

In [21]:
data_labels

array([0, 0, 0, ..., 4, 4, 4])

In [22]:
data_labels[data_labels==4] =1

In [23]:
set(data_labels)

{0, 1}

Tokenization

In [29]:
#
tokenizer = tfds.deprecated.text.SubwordTextEncoder.build_from_corpus(
    clean_tweet_train, target_vocab_size = 2**16
)

In [30]:
clean_tweet_train_input = [tokenizer.encode(sentence) for sentence in clean_tweet_train]

Padding

In [31]:
Max_len = max([len(sentence) for sentence in clean_tweet_train_input ])

In [34]:
clean_tweet_train_input = tf.keras.preprocessing.sequence.pad_sequences(
    clean_tweet_train_input,
    maxlen = Max_len,
    value = 0,
    padding = 'post'
)

In [35]:
test_idx = np.random.randint(0,80000,8000)

In [36]:
test_idx = np.concatenate(test_idx, test_idx+80000)

TypeError: ignored

In [ ]:
clean_tweet_train_input = clean_tweet_train_input[test_idx]
data_labels = data_labels[test_idx]
clean_tweet_train_input = np.delete(clean_tweet_train_input, test_idx, axis = 0)
data_labels = np.delete(data_labels, test_idx, axis = 0)